In [7]:
from helper_functions import load_data_for_all_three
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_lg")
from tqdm import tqdm, tqdm_notebook

In [8]:
text_data, labels = load_data_for_all_three('data')
df = pd.DataFrame(list(zip(text_data, labels)), columns=['text_data', 'label'])

Processing file: fitzgerald_all_the_sad.txt
Processing file: fitzgerald_bablyon_revisited.txt
Processing file: fitzgerald_beautiful_and_damned.txt
Processing file: fitzgerald_flappers_and_philosophers.txt
Processing file: fitzgerald_gatsby.txt
Processing file: fitzgerald_tales_jazz_age.txt
Processing file: fitzgerald_tender_is.txt
Processing file: fitzgerald_the_vegtable.txt
Processing file: fitzgerald_this_side.txt
Processing file: hemingway_across_the_river.txt
Processing file: hemingway_bell_tolls.txt
Processing file: hemingway_collected_stories.txt
Processing file: hemingway_farewell.txt
Processing file: hemingway_green_hills_africa.txt
Processing file: hemingway_in_our_time.txt
Processing file: hemingway_men_without_women.txt
Processing file: hemingway_old_man.txt
Processing file: hemingway_sun_also.txt
Processing file: hemingway_ten_poems.txt
Processing file: hemingway_winner_take_nothing.txt
Processing file: steinbeck_cannery_row.txt
Processing file: steinbeck_east_of_eden.txt
P

In [13]:
nlp.max_length = 2000000
# define a function to extract POS tags from a text and return a dictionary with the counts of each tag
def get_pos_counts(text):
    doc = nlp(text)
    pos_counts = {}
    for token in doc:
        pos = token.pos_
        if pos in pos_counts:
            pos_counts[pos] += 1
        else:
            pos_counts[pos] = 1
    return pos_counts

# apply the get_pos_counts function to each row in the dataframe
#df["pos_counts"] = df["text_data"].progress_apply(get_pos_counts)

def get_pos_counts_wrapper(text):
    return get_pos_counts(text)

df["pos_counts"] = list(tqdm(df["text_data"].apply(get_pos_counts_wrapper)))

# extract the POS tags and their counts into separate columns in the dataframe
pos_tags = set.union(*[set(d.keys()) for d in df["pos_counts"]])
for tag in pos_tags:
    df[tag] = df["pos_counts"].apply(lambda x: x.get(tag, 0))

df.head()

100%|██████████| 26/26 [00:00<00:00, 6522.24it/s]


,text_data,label,pos_counts,AUX,ADP,SYM,DET,PROPN,PUNCT,SCONJ,...,ADV,PART,ADJ,PRON,NUM,NOUN,VERB,SPACE,CCONJ,INTJ
0,"THE RICH BOY\n\n\nBegin with an individual, an...",0,"{'DET': 5854, 'ADJ': 4392, 'PROPN': 2910, 'SPA...",4394,7468,0,5854,2910,12974,1759,...,4157,1909,4392,9887,640,10788,9149,6041,2501,252
1,"""And where's Mr. Campbell?"" Charlie asked.\n\n...",0,"{'PUNCT': 1664, 'CCONJ': 307, 'SCONJ': 176, 'A...",509,820,0,609,397,1664,176,...,511,278,457,1247,62,1145,1103,279,307,42
2,CONTENTS\n\n\nBOOK ONE\n\n I. ANTHONY PATCH\n...,0,"{'NOUN': 22387, 'SPACE': 12102, 'PROPN': 6032,...",8366,14886,20,12086,6032,25383,3213,...,7716,3283,9878,17031,1424,22387,16909,12102,4885,664
3,Flappers and Philosophers\n\n\n\n\n\nThe Offsh...,0,"{'NOUN': 10596, 'CCONJ': 2525, 'PROPN': 3245, ...",4004,6781,2,5530,3245,13155,1478,...,4004,1712,4438,9109,661,10596,8966,6723,2525,338
4,"Then wear the gold hat, if that will move he...",0,"{'SPACE': 4745, 'ADV': 3129, 'VERB': 7301, 'DE...",2687,5810,8,4689,2350,10203,1245,...,3129,1349,3261,7494,438,8036,7301,4745,1940,172
